In [1]:
from ortools.sat.python import cp_model

## All-different values

In [4]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 2, 'x')
y = model.NewIntVar(0, 2, 'y')
z = model.NewIntVar(0, 2, 'z')

model.Add(x != y)
model.Add(y != z)
model.Add(x != z)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("x =", solver.Value(x))
    print("y =", solver.Value(y))
    print("z =", solver.Value(z))
else:
    print("No solution found.")

x = 2
y = 1
z = 0


## Simple Sum Constraint

In [19]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 10, 'x')
y = model.NewIntVar(0, 10, 'y')

model.Add(x + y == 10)

class AllSolutionsPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables: list[cp_model.IntVar]):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
    
    def OnSolutionCallback(self):
        self.__solution_count += 1
        print(f"Solution {self.__solution_count}: ", end="")
        for var in self.__variables:
            print(f"{var.Name()} = {self.Value(var)}", end=" ")
        print()

solver = cp_model.CpSolver()
solution_printer = AllSolutionsPrinter([x, y])
solver.SearchForAllSolutions(model, solution_printer)

Solution 1: x = 0 y = 10 
Solution 2: x = 1 y = 9 
Solution 3: x = 2 y = 8 
Solution 4: x = 3 y = 7 
Solution 5: x = 4 y = 6 
Solution 6: x = 5 y = 5 
Solution 7: x = 6 y = 4 
Solution 8: x = 7 y = 3 
Solution 9: x = 8 y = 2 
Solution 10: x = 9 y = 1 
Solution 11: x = 10 y = 0 


4

## Sudoku 4x4

In [21]:
model = cp_model.CpModel()

grid = [[model.NewIntVar(1, 4, f'cell_{i}_{j}') for j in range(4)] for i in range(4)]

for i in range(4):
    model.AddAllDifferent(grid[i])

for j in range(4):
    model.AddAllDifferent([grid[i][j] for i in range(4)])

for box_row in range(0, 4, 2):
    for box_col in range(0, 4, 2):
        box = [
            grid[box_row + i][box_col + j]
            for i in range(2)
            for j in range(2)
        ]
        model.AddAllDifferent(box)

class SudokuSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, grid_vars):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.grid_vars = grid_vars
        self.solution_count = 0

    def OnSolutionCallback(self):
        self.solution_count += 1
        print(f'\nSolution {self.solution_count}:')
        for row in self.grid_vars:
            print([self.Value(var) for var in row])

# Solve the model and search for all solutions
solver = cp_model.CpSolver()
solution_printer = SudokuSolutionPrinter(grid)
solver.SearchForAllSolutions(model, solution_printer)

print(f'\nTotal solutions found: {solution_printer.solution_count}')


Solution 1:
[3, 4, 1, 2]
[2, 1, 4, 3]
[4, 3, 2, 1]
[1, 2, 3, 4]

Solution 2:
[4, 3, 1, 2]
[2, 1, 4, 3]
[3, 4, 2, 1]
[1, 2, 3, 4]

Solution 3:
[4, 3, 1, 2]
[2, 1, 3, 4]
[3, 4, 2, 1]
[1, 2, 4, 3]

Solution 4:
[3, 4, 1, 2]
[2, 1, 3, 4]
[4, 3, 2, 1]
[1, 2, 4, 3]

Solution 5:
[4, 3, 1, 2]
[2, 1, 3, 4]
[3, 2, 4, 1]
[1, 4, 2, 3]

Solution 6:
[3, 4, 1, 2]
[2, 1, 4, 3]
[4, 2, 3, 1]
[1, 3, 2, 4]

Solution 7:
[4, 3, 1, 2]
[2, 1, 3, 4]
[1, 2, 4, 3]
[3, 4, 2, 1]

Solution 8:
[4, 3, 1, 2]
[1, 2, 3, 4]
[2, 1, 4, 3]
[3, 4, 2, 1]

Solution 9:
[4, 3, 1, 2]
[1, 2, 4, 3]
[2, 1, 3, 4]
[3, 4, 2, 1]

Solution 10:
[4, 3, 1, 2]
[2, 1, 4, 3]
[1, 2, 3, 4]
[3, 4, 2, 1]

Solution 11:
[3, 4, 1, 2]
[2, 1, 4, 3]
[1, 2, 3, 4]
[4, 3, 2, 1]

Solution 12:
[3, 4, 1, 2]
[1, 2, 4, 3]
[2, 1, 3, 4]
[4, 3, 2, 1]

Solution 13:
[3, 4, 1, 2]
[1, 2, 3, 4]
[2, 1, 4, 3]
[4, 3, 2, 1]

Solution 14:
[3, 4, 1, 2]
[2, 1, 3, 4]
[1, 2, 4, 3]
[4, 3, 2, 1]

Solution 15:
[3, 4, 1, 2]
[1, 2, 3, 4]
[4, 1, 2, 3]
[2, 3, 4, 1]

Solution 16:
[4, 3

## Job Scheduling

In [23]:
num_jobs = 3
num_machines = 2
durations = [3, 2, 4]
horizon = sum(durations)

model = cp_model.CpModel()

start = {}
end = {}
interval = {}
assigned = {}

for j in range(num_jobs):
    for m in range(num_machines):
        suffix = f"_j{j}_m{m}"

        assigned[j, m] = model.NewBoolVar(f'assigned{suffix}')
        start[j, m] = model.NewIntVar(0, horizon, f'start{suffix}')
        end[j, m] = model.NewIntVar(0, horizon, f'end{suffix}')

        interval[j, m] = model.NewOptionalIntervalVar(
            start[j, m], durations[j], end[j, m], assigned[j, m], f'interval{suffix}'
        )

for j in range(num_jobs):
    model.Add(sum(assigned[j, m] for m in range(num_machines)) == 1)

for m in range(num_machines):
    model.AddNoOverlap([interval[j, m] for j in range(num_jobs)])

makespan = model.NewIntVar(0, horizon, 'makespan')

job_ends = []
for j in range(num_jobs):
    job_end = model.NewIntVar(0, horizon, f'job{j}_end')
    model.AddMaxEquality(job_end, [end[j, m] for m in range(num_machines)])
    job_ends.append(job_end)

model.AddMaxEquality(makespan, job_ends)

model.Minimize(makespan)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print(f'Minimum makespan: {solver.Value(makespan)}\n')
    for j in range(num_jobs):
        for m in range(num_machines):
            if solver.Value(assigned[j, m]):
                print(f'Job {j} assigned to Machine {m}')
                print(f'  Start: {solver.Value(start[j, m])}')
                print(f'  End:   {solver.Value(end[j, m])}\n')
else:
    print('No solution found.')

Minimum makespan: 5

Job 0 assigned to Machine 1
  Start: 0
  End:   3

Job 1 assigned to Machine 1
  Start: 3
  End:   5

Job 2 assigned to Machine 0
  Start: 0
  End:   4



## Task Scheduling with Precedence Constraints

In [25]:
num_tasks = 4
durations = [3, 2, 2, 4]
horizon = sum(durations)

model = cp_model.CpModel()

tasks = {}

for t in range(num_tasks):
    start = model.NewIntVar(0, horizon, f'start_{t}')
    end = model.NewIntVar(0, horizon, f'end_{t}')
    interval = model.NewIntervalVar(start, durations[t], end, f'interval_{t}')
    tasks[t] = (start, end, interval)

model.Add(tasks[1][0] >= tasks[0][1])
model.Add(tasks[2][0] >= tasks[0][1])

model.Add(tasks[3][0] >= tasks[1][1])
model.Add(tasks[3][0] >= tasks[2][1])

makespan = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(makespan, [tasks[3][1]])
model.Minimize(makespan)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    for t in tasks:
        print(f"Task {t}: starts at {solver.value(tasks[t][0])}, ends at {solver.value(tasks[t][1])}")
    print(f"Total project duration (makespan): {solver.value(makespan)}")
else:
    print("No solution found.")

Task 0: starts at 0, ends at 3
Task 1: starts at 3, ends at 5
Task 2: starts at 3, ends at 5
Task 3: starts at 5, ends at 9
Total project duration (makespan): 9


## Software Build Pipeline Scheduling

In [29]:
model = cp_model.CpModel()

durations = {
    'A': 4,
    'B': 3,
    'C': 5,
    'D': 2,
    'E': 1,
    'F': 2
}

horizon = sum(durations.values())

tasks = {}

for t in durations:
    start = model.NewIntVar(0, horizon, f'start_{t}')
    end = model.NewIntVar(0, horizon, f'end_{t}')
    interval = model.NewIntervalVar(start, durations[t], end, f'interval_{t}')
    tasks[t] = (start, end, interval)

# A, B completed before C
model.Add(tasks['A'][1] <= tasks['C'][0])
model.Add(tasks['B'][1] <= tasks['C'][0])

# C, D completed before E
model.Add(tasks['C'][1] <= tasks['E'][0])
model.Add(tasks['D'][1] <= tasks['E'][0])

# E completed before F
model.Add(tasks['E'][1] <= tasks['F'][0])

makespan = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(makespan, [tasks['F'][1]])
model.Minimize(makespan)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    for t in tasks:
        print(f"Task {t}: starts at {solver.value(tasks[t][0])}, ends at {solver.value(tasks[t][1])}")
    print(f"Total project duration (makespan): {solver.value(makespan)}")
else:
    print("No solution found.")

Task A: starts at 0, ends at 4
Task B: starts at 0, ends at 3
Task C: starts at 4, ends at 9
Task D: starts at 0, ends at 2
Task E: starts at 9, ends at 10
Task F: starts at 10, ends at 12
Total project duration (makespan): 12


## Job Scheduling Problem

In [ ]:
durations = [4, 3, 5, 2, 3]
num_machines = 2
num_jobs = 5
horizon = sum(durations)

model = cp_model.CpModel()

start = {}
end = {}
interval = {}
assigned = {}

for j in range(num_jobs):
    for m in range(num_machines):
        suffix = f'_j{j}_m{m}'

        # Creates the variables in the dictionary for all start and end of every possible job machine combination
        start[j, m] = model.NewIntVar(0, horizon, f'start{suffix}')
        end[j, m] = model.NewIntVar(0, horizon, f'end{suffix}')
        assigned[j, m] = model.NewBoolVar(f'assigned{suffix}')

        # Creates an interval variable for every possible job machine combination
        interval[j, m] = model.NewOptionalIntervalVar(
            start[j, m], durations[j], end[j, m], assigned[j, m], f'interval{suffix}'
        )

# Add the condition that for job[i] only 1 machine is assigned to it. So sum should be 1. (Remember this is a constraint being added)
for j in range(num_jobs):
    model.Add(sum(assigned[j, m] for m in range(num_machines)) == 1)

# Add the condition that for machine[i] no job interval overlaps
for m in range(num_machines):
    model.AddNoOverlap([interval[j, m] for j in range(num_jobs)])

# Need to minimize the makespan, so created its variables
makespan = model.NewIntVar(0, horizon, 'makespan')

# This will store all the max job ends
job_ends = []

for j in range(num_jobs):
    # Variable to store a max job end
    job_end = model.NewIntVar(0, horizon, f'end_{j}')
    # job_end variable is to be max from all end times for every machine for a job[i]
    model.AddMaxEquality(job_end, [end[j, m] for m in range(num_machines)])
    job_ends.append(job_end)

# Get the max job_end from the array and have makespan be that
model.AddMaxEquality(makespan, job_ends)

# Make makespan minimum 
# Commented Out Because For Optimization Problem The Class Below Doesnt Work
# model.Minimize(makespan)

class AllSchedulingPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, assigned, start, end, num_jobs, num_machines, makespan):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.assigned = assigned
        self.start = start
        self.end = end
        self.num_jobs = num_jobs
        self.num_machines = num_machines
        self.makespan = makespan
        self.solution_count = 0

    def OnSolutionCallback(self):
        self.solution_count += 1
        if self.solution_count >= 10:
            self.StopSearch()
        print(f"\nSolution #{self.solution_count}")
        print(f"Makespan {self.Value(self.makespan)}")
        for j in range(self.num_jobs):
            for m in range(self.num_machines):
                if self.Value(self.assigned[j, m]):
                    s = self.Value(self.start[j, m])
                    e = self.Value(self.end[j, m])
                    print(f"  Job {j} -> Machine {m} | Start: {s}, End: {e}")

solver = cp_model.CpSolver()
status = solver.solve(model)

printer = AllSchedulingPrinter(assigned, start, end, num_jobs, num_machines, makespan)
solver.parameters.enumerate_all_solutions = True
status = solver.SearchForAllSolutions(model, printer)



Solution #1
Makespan 17
  Job 0 -> Machine 1 | Start: 5, End: 9
  Job 1 -> Machine 1 | Start: 9, End: 12
  Job 2 -> Machine 1 | Start: 0, End: 5
  Job 3 -> Machine 1 | Start: 15, End: 17
  Job 4 -> Machine 1 | Start: 12, End: 15

Solution #2
Makespan 17
  Job 0 -> Machine 1 | Start: 0, End: 4
  Job 1 -> Machine 1 | Start: 4, End: 7
  Job 2 -> Machine 1 | Start: 10, End: 15
  Job 3 -> Machine 1 | Start: 15, End: 17
  Job 4 -> Machine 1 | Start: 7, End: 10

Solution #3
Makespan 17
  Job 0 -> Machine 1 | Start: 0, End: 4
  Job 1 -> Machine 1 | Start: 4, End: 7
  Job 2 -> Machine 1 | Start: 10, End: 15
  Job 3 -> Machine 1 | Start: 15, End: 17
  Job 4 -> Machine 1 | Start: 7, End: 10

Solution #4
Makespan 17
  Job 0 -> Machine 1 | Start: 0, End: 4
  Job 1 -> Machine 1 | Start: 4, End: 7
  Job 2 -> Machine 1 | Start: 10, End: 15
  Job 3 -> Machine 1 | Start: 15, End: 17
  Job 4 -> Machine 1 | Start: 7, End: 10

Solution #5
Makespan 17
  Job 0 -> Machine 1 | Start: 0, End: 4
  Job 1 -> Mach